In [7]:
#TO RE-RUN
%reset -f

In [8]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from datetime import datetime
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

import sys
sys.path.insert(1, "../src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter
import MLpipeline as MLpipeline

#### Run experiments

In [21]:
typeEncounter = "last" # ['first','last']
typeHypothesis = "early_readmission_vs_none" # ['all_readmisssion_vs_none','early_readmission_vs_none']
typeDataFeatures = "extended_extra" # ["non_extended","extended','extended_extra']
    #Extended -> Subset of columns
    #Minimum -> minimum set of columns 
typeDiagnosis = "diag_1"  #["none","diag_1", "diag_3"]    
typeDisease = "subset" # ["subset","any",["Respiratory",...]]
    #subset -> Return subset of predefined disease features
    #any -> Return all disease features    
    #disease -> Return diseases feature
typeDataExperiment = "all" #["all", "disease"] 
    #all -> Include all diagnosis as columns
    #disease -> Remove diagnosis as column and keep only rows with diagnosis == 1       

In [22]:
verbose = True
cv_thr = 0.3
cv_folds = 5

tr_thrs = [1.0] # [0.1,0.2,0.4,0.6,1.0]
ts_thr = 0.30

fs_methods = ["none",] #["none","combine_fs","lasso_fs","rfe_rf_fs"]
cls_methods = ["rf","logReg","knn","nb"] #["rf","svmRBF","logReg","knn","nn","gbt"]
lms = ["recall","f1","f1_weighted"] #["f1_weighted","precision_weighted","roc_auc","recall"]
sm_types = ["none"] #["none","after"]
sm_method = "sm_smote"

In [23]:
#Load data
df_all = MLpipeline.load_data(typeEncounter, typeDiagnosis, typeDataFeatures)
print "\nSHAPE:"
print df_all.shape
print "\nInitial columns:"
print df_all.columns

#Filter data by class
df_all = MLpipeline.filter_data_by_class(df_all, typeHypothesis)
print "\nRows by class type:"
print df_all.iloc[:,-1].sort_values().unique(), np.sum(df_all["readmitted"] == 0), np.sum(df_all["readmitted"] == 1)
    
#Get columns
colsDiseases, colsNonDiseases = MLpipeline.get_columns(df_all,typeDiagnosis)
print "\nDiseases:", colsDiseases
print "\nNon-diseases:", colsNonDiseases
    
#Load diseases
diseases = MLpipeline.get_diseases(colsDiseases, typeDisease)
print "\nTotal data:", df_all.shape
print diseases


SHAPE:
(67182, 69)

Initial columns:
Index([u'gender', u'age', u'race_AfricanAmerican', u'race_Caucasian',
       u'race_Other', u'HbA1c', u'Change', u'time_in_hospital', u'diabetesMed',
       u'diss_home', u'medSpec_cardio', u'medSpec_Family/GeneralPractice',
       u'medSpec_InternalMedicine', u'medSpec_surgery', u'adm_src_1',
       u'adm_src_2', u'adm_src_3', u'adm_src_4', u'adm_src_5', u'adm_src_6',
       u'adm_src_7', u'adm_src_8', u'adm_src_10', u'adm_src_11', u'adm_src_13',
       u'adm_src_14', u'adm_src_22', u'adm_src_25', u'adm_1', u'adm_2',
       u'adm_3', u'adm_4', u'adm_7', u'number_treatment',
       u'num_lab_procedures', u'num_procedures', u'num_medications',
       u'number_outpatient', u'number_emergency', u'number_inpatient',
       u'number_diagnoses', u'insulin', u'metformin', u'pioglitazone',
       u'glimepiride', u'glipizide', u'repaglinide', u'nateglinide',
       u'ComplexHbA1c', u'add_in_out', u'add_procs_meds', u'div_visits_time',
       u'div_em_time',

In [ ]:
res = MLpipeline.run(df_all, colsNonDiseases, diseases, typeDataFeatures, 
               typeDataExperiment, typeEncounter, typeHypothesis, typeDiagnosis, ts_thr, tr_thrs, 
               fs_methods, sm_method, sm_types, 
               cls_methods, lms, cv_folds, cv_thr, True, True)

(12, 6)

DataSet:
**********
**********
SIZE: 1.0
DISEASE: subset
(45779, 69)
ALL TRAIN: (32045, 68)
TRAIN: [0's: 27849 1's: 4196 ]
ALL TEST: (13734, 68)
TEST: [0's: 11936 1's: 1798 ]

Num experiment: 0 / 11
****************
FS: none
SM: none
CLS: rf
METRIC: recall
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  3.7min


In [12]:
if 'dfAux' not in locals():
    print "Not dfAux"
    dfAux = pd.DataFrame()

if 'df' in locals():
    print "Exist df"
    dfAux = df.copy()
    
df = pd.DataFrame(np.array(res).reshape(len(res),27), columns=
                          ["exp", "typeDisease","typeEncounter","typeHypothesis","typeDataFeatures","typeDiagnosis",
                           "size_tr","fs","sm","cls","metric","params",
                           "tr_f1","tr_prec","tr_rec",
                           "cv_f1_mean","cv_f1_std","cv_prec_mean","cv_prec_std","cv_rec_mean","cv_rec_std",
                           "test_f1","test_prec","test_rec","test_auc",
                           "time","pipeline"])

df[["size_tr","fs","sm","metric","cls","params", 'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 'test_f1', 'test_prec', 'test_rec', 'test_auc']]

Exist df


,size_tr,fs,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.01,combine_fs,after,f1,logReg,"[None, 0.0005, l2, 40, 3]",0.62,0.62,0.62,0.56,0.56,0.56,0.59,0.59,0.59,0.58
1,0.01,combine_fs,after,f1,rf,"[gini, 12, 250, 30, 3]",0.64,0.68,0.67,0.55,0.55,0.57,0.57,0.61,0.59,0.55
2,0.01,combine_fs,after,recall,rf,"[gini, 12, 250, 30, 3]",0.64,0.68,0.67,0.55,0.55,0.57,0.57,0.61,0.59,0.55
3,0.01,combine_fs,none,f1,nb,[50],0.61,0.61,0.62,0.55,0.56,0.58,0.59,0.60,0.59,0.57
4,0.01,combine_fs,none,recall,nb,[30],0.58,0.65,0.64,0.57,0.60,0.61,0.56,0.61,0.60,0.55
5,0.01,combine_fs,none,f1,knn,"[uniform, 3, 30]",0.62,0.62,0.63,0.45,0.51,0.51,0.54,0.56,0.54,0.52
6,0.01,combine_fs,none,recall,knn,"[uniform, 3, 30]",0.62,0.62,0.63,0.45,0.51,0.51,0.54,0.56,0.54,0.52
7,0.01,combine_fs,none,f1,logReg,"[balanced, 0.005, l2, 40]",0.61,0.61,0.61,0.57,0.57,0.57,0.59,0.59,0.59,0.58
8,0.01,combine_fs,none,recall,logReg,"[balanced, 1e-05, l2, 50]",0.62,0.63,0.61,0.56,0.57,0.56,0.58,0.57,0.59,0.57
9,0.01,combine_fs,none,f1,rf,"[entropy, None, 500, 30]",0.64,0.68,0.67,0.55,0.55,0.57,0.57,0.61,0.60,0.56


In [15]:
dfAux =pd.DataFrame()
df = pd.concat((dfAux, df))
df[["size_tr","sm","fs","metric","cls","params",'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 'test_f1', 'test_prec', 
        'test_rec', 'test_auc','time']].sort_values("time", ascending=True)

,size_tr,sm,fs,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc,time
35,0.01,none,none,f1,nb,[],0.63,0.63,0.64,0.56,0.57,0.58,0.58,0.59,0.58,0.56,2.25
34,0.01,none,none,recall,nb,[],0.63,0.63,0.64,0.56,0.57,0.58,0.58,0.59,0.58,0.56,2.57
29,0.01,after,none,f1,nb,[4],0.60,0.66,0.60,0.52,0.56,0.54,0.51,0.52,0.57,0.55,2.59
42,0.01,after,none,recall,nb,[4],0.60,0.66,0.60,0.52,0.56,0.54,0.51,0.52,0.57,0.55,2.71
21,0.01,none,lasso_fs,f1,nb,[1],0.63,0.63,0.64,0.56,0.59,0.59,0.58,0.59,0.58,0.56,2.91
4,0.01,none,combine_fs,recall,nb,[30],0.58,0.65,0.64,0.57,0.60,0.61,0.56,0.61,0.60,0.55,2.91
20,0.01,none,lasso_fs,recall,nb,[1],0.63,0.63,0.64,0.56,0.59,0.59,0.58,0.59,0.58,0.56,2.94
3,0.01,none,combine_fs,f1,nb,[50],0.61,0.61,0.62,0.55,0.56,0.58,0.59,0.60,0.59,0.57,2.95
27,0.01,after,lasso_fs,f1,nb,"[1, 4]",0.56,0.63,0.57,0.51,0.57,0.54,0.50,0.51,0.57,0.54,3.59
26,0.01,after,lasso_fs,recall,nb,"[1, 4]",0.56,0.63,0.57,0.51,0.57,0.54,0.50,0.51,0.57,0.54,3.60
